In [1]:
!pip install torch torchvision torchaudio
!pip install ogb
!pip install torch-geometric

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 80.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
import os, random, numpy as np, torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.utils import to_networkx
import networkx as nx
import wandb
from kaggle_secrets import UserSecretsClient

# W&B login via Kaggle secrets
user_secrets = UserSecretsClient()
wandb.login(key=user_secrets.get_secret("wandb-key"))

EPOCHS = int(os.getenv("EPOCHS", "50"))
NUM_SEEDS = int(os.getenv("NUM_SEEDS", "30"))
PROJECT = "GCN Graph classification(IMDB-BINARY)_Gal"
ENTITY = "noynetanel1-tel-aviv-university"

def set_seed(s):
    random.seed(s)
    np.random.seed(s)
    torch.manual_seed(s)
    torch.cuda.manual_seed_all(s)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

class CentralityEncoder(nn.Module):
    def __init__(self, in_dim=1, out_dim=16):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, 32),
            nn.ReLU(),
            nn.Linear(32, out_dim)
        )
    def forward(self, c):
        return self.net(c)

class GCN(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, cent_dim=16):
        super().__init__()
        self.cent_enc = CentralityEncoder(1, cent_dim)
        self.conv1 = GCNConv(in_channels + cent_dim, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.lin = nn.Linear(hidden_channels, out_channels)
    def forward(self, x, cent, edge_index, batch):
        x = torch.cat([x, self.cent_enc(cent)], dim=1)
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = global_mean_pool(x, batch)
        return F.log_softmax(self.lin(x), dim=1)

def _minmax_norm(t):
    if len(t) == 0:
        return t
    vmin, vmax = t.min(), t.max()
    if (vmax - vmin) > 0:
        return (t - vmin) / (vmax - vmin)
    return torch.zeros_like(t)

def make_centrality_transform():
    def transform(data):
        if getattr(data, "x", None) is None:
            data.x = torch.ones((data.num_nodes, 1), dtype=torch.float)
        G = to_networkx(data, to_undirected=True)
        cent = torch.zeros((data.num_nodes, 1), dtype=torch.float)
        if G.number_of_nodes() > 0:
            try:
                ev = nx.eigenvector_centrality_numpy(G)
            except Exception:
                ev = {n: 0.0 for n in G.nodes}
            ev_tensor = torch.tensor([ev.get(n, 0.0) for n in range(data.num_nodes)], dtype=torch.float)
            cent[:, 0] = _minmax_norm(ev_tensor)
        data.cent = cent
        return data
    return transform

def run_seed(seed):
    set_seed(seed)
    dataset = TUDataset(root="data/TUDataset", name="IMDB-BINARY", transform=make_centrality_transform())
    dataset = dataset.shuffle()
    train_dataset = dataset[:len(dataset)//2]
    test_dataset = dataset[len(dataset)//2:]
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = GCN(dataset.num_features if dataset.num_features else 1, 64, dataset.num_classes).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    wandb.init(
        project=PROJECT,
        entity=ENTITY,
        name=f"GCN_Centrality_PROTEINS_seed_{seed}",
        group="GCN_PROTEINS_Centrality",
        job_type="centrality_only",
        reinit=True,
        config={
            "model": "GCN",
            "hidden_channels": 64,
            "epochs": EPOCHS,
            "lr": 0.001,
            "dataset": "PROTEINS",
            "task": "Centrality_eigenvector",
            "seed": seed
        }
    )

    def train():
        model.train()
        total_loss, correct, count = 0.0, 0, 0
        for data in train_loader:
            data = data.to(device)
            optimizer.zero_grad()
            out = model(data.x, data.cent, data.edge_index, data.batch)
            loss = F.nll_loss(out, data.y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            pred = out.argmax(dim=1)
            correct += int((pred == data.y).sum())
            count += data.num_graphs
        return total_loss / len(train_loader), correct / count

    def test(loader):
        model.eval()
        correct, count = 0, 0
        for data in loader:
            data = data.to(device)
            out = model(data.x, data.cent, data.edge_index, data.batch)
            pred = out.argmax(dim=1)
            correct += int((pred == data.y).sum())
            count += data.num_graphs
        return correct / count

    for epoch in range(1, EPOCHS + 1):
        loss, train_acc = train()
        test_acc = test(test_loader)
        print(f"Seed {seed} | Epoch {epoch:03d} | Loss: {loss:.4f} | Train Acc: {train_acc:.4f} | Test Acc: {test_acc:.4f}")
        wandb.log({
            "epoch": epoch,
            "loss": loss,
            "train_acc": train_acc,
            "test_acc": test_acc
        })

    wandb.finish()

if __name__ == "__main__":
    for s in range(1, NUM_SEEDS + 1):
        run_seed(s)


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: galsalman (noynetanel1-tel-aviv-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
Processing...
Done!
wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.
wandb: Tracking run with wandb version 0.20.1
wandb: Run data is saved locally in /kaggle/working/wandb/run-20250914_174746-b4t5soyi
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run GCN_Centrality_PROTEINS_seed_1
wandb: ⭐️ View project at https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28IMDB-BINARY%29_Gal
wandb: 🚀 View run at https://w

Seed 1 | Epoch 001 | Loss: 0.6966 | Train Acc: 0.4720 | Test Acc: 0.4920
Seed 1 | Epoch 002 | Loss: 0.6939 | Train Acc: 0.5080 | Test Acc: 0.4920
Seed 1 | Epoch 003 | Loss: 0.6916 | Train Acc: 0.5080 | Test Acc: 0.4540
Seed 1 | Epoch 004 | Loss: 0.6908 | Train Acc: 0.5260 | Test Acc: 0.4920
Seed 1 | Epoch 005 | Loss: 0.6920 | Train Acc: 0.5380 | Test Acc: 0.4980
Seed 1 | Epoch 006 | Loss: 0.6892 | Train Acc: 0.5380 | Test Acc: 0.5000
Seed 1 | Epoch 007 | Loss: 0.6864 | Train Acc: 0.5560 | Test Acc: 0.4740
Seed 1 | Epoch 008 | Loss: 0.6869 | Train Acc: 0.5300 | Test Acc: 0.4920
Seed 1 | Epoch 009 | Loss: 0.6845 | Train Acc: 0.5760 | Test Acc: 0.5140
Seed 1 | Epoch 010 | Loss: 0.6842 | Train Acc: 0.5660 | Test Acc: 0.4960
Seed 1 | Epoch 011 | Loss: 0.6846 | Train Acc: 0.5700 | Test Acc: 0.5100
Seed 1 | Epoch 012 | Loss: 0.6834 | Train Acc: 0.5920 | Test Acc: 0.4780
Seed 1 | Epoch 013 | Loss: 0.6830 | Train Acc: 0.5580 | Test Acc: 0.5140
Seed 1 | Epoch 014 | Loss: 0.6843 | Train Acc: 0.57

wandb: uploading output.log; uploading config.yaml
wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
wandb:      loss █▇▇▇▇▆▆▅▅▅▅▅▅▅▄▄▄▄▄▅▄▃▄▃▃▃▄▃▂▃▂▂▂▁▁▂▂▁▁▂
wandb:  test_acc ▃▃▁▃▄▂▃▅▄▅▆▃▄▄▄▅▄▆▃▅▅▄▅▆▆▆▆██▇██▆▇▆▆▆▅▇█
wandb: train_acc ▁▃▃▅▅▄▇▆█▆▇▇▆▆▇▇▇▄▇▆▆▆▆▇▇▇█▇▆▇▆▇▆▇▇▇▇▇▇▆
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 0.6719
wandb:  test_acc 0.562
wandb: train_acc 0.57
wandb: 
wandb: 🚀 View run GCN_Centrality_PROTEINS_seed_1 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28IMDB-BINARY%29_Gal/runs/b4t5soyi
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28IMDB-BINARY%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250914_174746-b4t5soyi/logs
wandb: Tracking run with wandb versio

Seed 2 | Epoch 001 | Loss: 0.6964 | Train Acc: 0.4960 | Test Acc: 0.5240
Seed 2 | Epoch 002 | Loss: 0.6915 | Train Acc: 0.5280 | Test Acc: 0.4860
Seed 2 | Epoch 003 | Loss: 0.6895 | Train Acc: 0.5320 | Test Acc: 0.4940
Seed 2 | Epoch 004 | Loss: 0.6895 | Train Acc: 0.5700 | Test Acc: 0.5300
Seed 2 | Epoch 005 | Loss: 0.6882 | Train Acc: 0.5620 | Test Acc: 0.5180
Seed 2 | Epoch 006 | Loss: 0.6862 | Train Acc: 0.5700 | Test Acc: 0.5220
Seed 2 | Epoch 007 | Loss: 0.6856 | Train Acc: 0.5520 | Test Acc: 0.5240
Seed 2 | Epoch 008 | Loss: 0.6837 | Train Acc: 0.5640 | Test Acc: 0.5200
Seed 2 | Epoch 009 | Loss: 0.6836 | Train Acc: 0.5820 | Test Acc: 0.5260
Seed 2 | Epoch 010 | Loss: 0.6839 | Train Acc: 0.5780 | Test Acc: 0.5180
Seed 2 | Epoch 011 | Loss: 0.6828 | Train Acc: 0.5640 | Test Acc: 0.5200
Seed 2 | Epoch 012 | Loss: 0.6858 | Train Acc: 0.5720 | Test Acc: 0.5240
Seed 2 | Epoch 013 | Loss: 0.6845 | Train Acc: 0.5600 | Test Acc: 0.5160
Seed 2 | Epoch 014 | Loss: 0.6837 | Train Acc: 0.57

wandb: uploading output.log; uploading config.yaml
wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
wandb:      loss █▇▇▇▆▆▆▆▆▅▆▅▅▅▅▄▅▄▄▄▄▃▃▃▂▄▅▅▂▂▂▁▁▂▃▂▁▁▂▂
wandb:  test_acc ▄▁▂▄▃▄▃▄▄▃▄▄▄▄▄▄▅▄▄▄▄▆▄▇▃▇▅▄▇▆▆▇▇███▇▇█▇
wandb: train_acc ▁▃▃▆▅▅▅▆▆▅▅▆▄▆▆▆▆▆▆▅▆▆▅▇▇▇▇▆▅▆▇▇▇▇▆▇██▇▇
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 0.66401
wandb:  test_acc 0.568
wandb: train_acc 0.588
wandb: 
wandb: 🚀 View run GCN_Centrality_PROTEINS_seed_2 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28IMDB-BINARY%29_Gal/runs/6gtjx6sc
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28IMDB-BINARY%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250914_174913-6gtjx6sc/logs
wandb: Tracking run with wandb vers

...

wandb: uploading wandb-summary.json; uploading config.yaml
wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
wandb:      loss ███▇▇█▇▇▇▇▇▇▆▆▆▆▆▆▅▆▄▅▄▄▄▄▃▃▃▃▂▃▃▂▅▂▂▂▁▁
wandb:  test_acc ▃▃▁▄▃▃▂▂▅▃▄▃▅▃▅▄▃▅▅▄▅▄▅▅▇█▆▅▇▅▆▅▆▄▇█▇▇▇▇
wandb: train_acc ▃▃▄▃▄▁▅▆▃▅▄▆▆▅▇▅▅▅▆▆▆█▆▇▇█▆▆▆█▆▇█▅███▇▇█
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 0.66288
wandb:  test_acc 0.566
wandb: train_acc 0.608
wandb: 
wandb: 🚀 View run GCN_Centrality_PROTEINS_seed_28 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28IMDB-BINARY%29_Gal/runs/mm23jad6
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28IMDB-BINARY%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250914_182805-mm23jad6/logs
wandb: Tracking run with w

Seed 29 | Epoch 001 | Loss: 0.6981 | Train Acc: 0.4700 | Test Acc: 0.5080
Seed 29 | Epoch 002 | Loss: 0.6929 | Train Acc: 0.5060 | Test Acc: 0.4920
Seed 29 | Epoch 003 | Loss: 0.6918 | Train Acc: 0.5160 | Test Acc: 0.5300
Seed 29 | Epoch 004 | Loss: 0.6921 | Train Acc: 0.5080 | Test Acc: 0.5120
Seed 29 | Epoch 005 | Loss: 0.6918 | Train Acc: 0.5160 | Test Acc: 0.5440
Seed 29 | Epoch 006 | Loss: 0.6896 | Train Acc: 0.5660 | Test Acc: 0.5080
Seed 29 | Epoch 007 | Loss: 0.6916 | Train Acc: 0.5240 | Test Acc: 0.5440
Seed 29 | Epoch 008 | Loss: 0.6901 | Train Acc: 0.5540 | Test Acc: 0.5260
Seed 29 | Epoch 009 | Loss: 0.6879 | Train Acc: 0.5620 | Test Acc: 0.5040
Seed 29 | Epoch 010 | Loss: 0.6889 | Train Acc: 0.5700 | Test Acc: 0.5360
Seed 29 | Epoch 011 | Loss: 0.6888 | Train Acc: 0.5740 | Test Acc: 0.5220
Seed 29 | Epoch 012 | Loss: 0.6894 | Train Acc: 0.5520 | Test Acc: 0.5040
Seed 29 | Epoch 013 | Loss: 0.6886 | Train Acc: 0.5520 | Test Acc: 0.5480
Seed 29 | Epoch 014 | Loss: 0.6881 | T

wandb: uploading output.log; uploading config.yaml
wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
wandb:      loss █▆▆▆▆▆▅▅▅▅▅▅▅▅▄▅▅▅▄▄▅▅▄▄▅▄▄▃▄▄▃▄▄▄▂▂▂▁▃▁
wandb:  test_acc ▃▁▅▆▃▄▂▅▄▂▂▆▃▄▂▆▄▇▄▄▄▄▄▅▄▆▅▅▅▆▅▄▅▇▅▇▇▅▇█
wandb: train_acc ▁▃▄▃▄▄▆▆▇▇▆▆▇▆▆▆▇▇▆▇▆▆▇▆▇▆▆▆▇▆▆▇▆▇▅▆▇█▆▇
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 0.67445
wandb:  test_acc 0.566
wandb: train_acc 0.57
wandb: 
wandb: 🚀 View run GCN_Centrality_PROTEINS_seed_29 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28IMDB-BINARY%29_Gal/runs/yp2q2ob1
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28IMDB-BINARY%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250914_182932-yp2q2ob1/logs
wandb: Tracking run with wandb vers

Seed 30 | Epoch 001 | Loss: 0.6947 | Train Acc: 0.4980 | Test Acc: 0.5380
Seed 30 | Epoch 002 | Loss: 0.6942 | Train Acc: 0.5000 | Test Acc: 0.4900
Seed 30 | Epoch 003 | Loss: 0.6950 | Train Acc: 0.5080 | Test Acc: 0.4920
Seed 30 | Epoch 004 | Loss: 0.6930 | Train Acc: 0.5000 | Test Acc: 0.5720
Seed 30 | Epoch 005 | Loss: 0.6930 | Train Acc: 0.5040 | Test Acc: 0.4920
Seed 30 | Epoch 006 | Loss: 0.6941 | Train Acc: 0.4900 | Test Acc: 0.4900
Seed 30 | Epoch 007 | Loss: 0.6932 | Train Acc: 0.5040 | Test Acc: 0.4920
Seed 30 | Epoch 008 | Loss: 0.6926 | Train Acc: 0.5080 | Test Acc: 0.4920
Seed 30 | Epoch 009 | Loss: 0.6923 | Train Acc: 0.5220 | Test Acc: 0.5700
Seed 30 | Epoch 010 | Loss: 0.6929 | Train Acc: 0.5300 | Test Acc: 0.5500
Seed 30 | Epoch 011 | Loss: 0.6913 | Train Acc: 0.5000 | Test Acc: 0.4920
Seed 30 | Epoch 012 | Loss: 0.6915 | Train Acc: 0.5080 | Test Acc: 0.4900
Seed 30 | Epoch 013 | Loss: 0.6916 | Train Acc: 0.5280 | Test Acc: 0.5600
Seed 30 | Epoch 014 | Loss: 0.6946 | T

wandb: uploading output.log; uploading config.yaml; uploading history steps 44-49, summary, console lines 44-49
wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
wandb:      loss ███▇▇█▇▇▇▇▇█▇▆▆▆▅▅▆▅▄▄▄▃▃▃▃▅▄▃▄▂▂▂▂▃▁▁▂▃
wandb:  test_acc ▄▁▁▆▁▁▁▆▅▁▅▅▄▅▅▄▇▇▇▇█▇▇▇▇▇▆█▅█▇█▇█▇▇█▆▇▇
wandb: train_acc ▂▂▃▂▂▂▃▄▄▂▄▁▆▆▆▇▆▆█▇▇▆▆▆▇▆▇█▇▆▇▆▇█▇████▇
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 0.67634
wandb:  test_acc 0.582
wandb: train_acc 0.576
wandb: 
wandb: 🚀 View run GCN_Centrality_PROTEINS_seed_30 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28IMDB-BINARY%29_Gal/runs/sbcgtjgm
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28IMDB-BINARY%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-202